In [ ]:
import sys
import os
import pandas as pd
import datetime

sys.path.append(os.path.abspath(os.path.join('..')))

from src.data_loader import fetch_data_tiingo

# Config block
START_DATE = "2009-01-01"
END_DATE = datetime.datetime.today().strftime('%Y-%m-%d')
API_KEY = "08ec876fa0ec9c87cafff1fd07c6ffd95036203a"
ticker = "XLE"
data_path = f"../data/{ticker.lower()}_{START_DATE}_{END_DATE}.csv"

if os.path.exists(data_path):
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
else:
    df = fetch_data_tiingo(ticker, api_key=API_KEY, start_date=START_DATE, end_date=END_DATE)
    
    #Save for local use
    os.makedirs("../data", exist_ok=True)
    df.to_csv(data_path)

In [3]:
import sys
import os
import pandas as pd

# Allow importing from ../src
sys.path.append(os.path.abspath(os.path.join("..")))

from src.strategy import sma_crossover_strategy
from src.backtester import backtest
from src.metrics import sharpe_ratio, max_drawdown
from src.utils import plot_equity_curve

# Load the saved data
df = pd.read_csv("../data/xle_2009-01-01_2025-07-28.csv", index_col=0, parse_dates=True)

df_raw = df.copy()

# Apply strategy and run backtest
df = sma_crossover_strategy(df, short_window=20, long_window=50)
results = backtest(df)

# Compute metrics
sr = sharpe_ratio(results['Strategy_Returns'].dropna())
dd = max_drawdown(results['Equity_Curve'].dropna())


print(f"Sharpe Ratio: {sr:.2f}")
print(f"Max Drawdown: {dd:.2%}")

# Plot equity curve
plot_equity_curve(results)

Sharpe Ratio: -0.33
Max Drawdown: -90.41%


We applied a simple 20/50-day Simple Moving Average (SMA) crossover strategy to XLE, the Energy Select Sector SPDR ETF, covering the period from 2009 to July 2025. This strategy generates long-only signals when the short-term SMA (20-day) crosses above the long-term SMA (50-day).

Key performance metrics:
Sharpe Ratio: –0.31
Indicates that the strategy delivered negative risk-adjusted returns. It underperformed even a zero-return (risk-free) benchmark, on a volatility-adjusted basis.

Max Drawdown: –89.45%
The strategy suffered a near-complete loss of capital at its worst point, highlighting poor downside protection and high vulnerability to market noise.

Equity curve analysis:
The equity curve shows a steady and persistent drawdown from 2009 onward, with only short-lived recoveries. There are multiple failed breakouts and whipsaw periods where the SMA crossover generated false signals, resulting in frequent losses and erosion of capital.

Notably, there is no sustained upward trend in cumulative returns. Sharp drops (e.g., during 2015 oil crash, 2020 COVID, 2022 energy volatility) caused deep drawdowns. The curve shows volatility without reward a hallmark of a strategy out of sync with market structure.

Takeaways:
An obvious point, not all strategies generalise across all asset classes. SMA crossovers work better on trending assets (e.g., SPY, NASDAQ, BTC) than on cyclicals or commodities.
Backtesting is Discovery: Negative results are not failures. They help us reject poor hypotheses and avoid overfitting.
Comparing across assets, adding filters (e.g., volatility regimes), or hybrid signals may improve strategy viability.

Let us put to test the third takeaway. 

Adding a volatility filter: Don't trade below unless volatility is below a certain threshold (to avoid whipsaws).

Idea: Use Average True Range (ATR) or rolling standard deviation as a proxy for volatility. Only enter trades when volatility is below a defined percentile (eg: 50th percentile). 

Intuition:
High volatility = unstable signals → avoid
Low volatility = more stable regime → allow SMA crossover to execute

In [5]:
import sys
import os
import pandas as pd

# Allow importing from ../src
sys.path.append(os.path.abspath(os.path.join("..")))

from src.strategy import sma_crossover_strategy, add_atr_filter, sma_crossover_with_atr_filter
from src.backtester import backtest
from src.metrics import sharpe_ratio, sortino_ratio, max_drawdown, calmar_ratio, trade_statistics
from src.utils import plot_equity_curve, plot_comparison_equity_curves
#from src.debug_tools import analyze_trades
    
# Load the saved data
df = pd.read_csv("../data/xle_2009-01-01_2025-07-28.csv", index_col=0, parse_dates=True)

df_raw = df.copy()

# 1. Add volatility info
df_atr = add_atr_filter(df_raw.copy())

# 2. Naive SMA strategy
df_naive = sma_crossover_strategy(df_atr.copy())
results_naive = backtest(df_naive)

# 3. ATR-filtered SMA strategy
df_filtered = sma_crossover_with_atr_filter(df_atr.copy())
results_filtered = backtest(df_filtered)

# 4. Metrics Function
def summarize_metrics(results, label):
    sr = sharpe_ratio(results['Strategy_Returns'].dropna())
    so = sortino_ratio(results['Strategy_Returns'].dropna())
    dd = max_drawdown(results['Equity_Curve'].dropna())
    cr = calmar_ratio(results['Strategy_Returns'].dropna(), results['Equity_Curve'].dropna())
    ts = trade_statistics(results['Signal'], results['Returns'])

    print(f"\n📊 {label} Strategy Metrics")
    print(f"Sharpe Ratio: {sr:.2f}")
    print(f"Sortino Ratio: {so:.2f}")
    print(f"Calmar Ratio: {cr:.2f}")
    print(f"Max Drawdown: {dd:.2%}")
    print(f"Total Trades: {ts['Number of Trades']}, Win Rate: {ts['Win Rate']:.1%}")
    print(f"Avg Profit (Winners): {ts['Average Win']:.2%}, Avg Loss (Losers): {ts['Average Loss']:.2%}")

# 5. Show all metrics
summarize_metrics(results_naive, "Naive SMA")
summarize_metrics(results_filtered, "Volatility-Aware SMA")

plot_comparison_equity_curves({
    "Naive SMA": results_naive,
    "ATR-Filtered SMA": results_filtered,
    "Buy & Hold": results_naive.assign(Equity_Curve=results_naive['Benchmark'])  # Trick to reuse plot
})

c:\Users\yaswa\OneDrive\Documents\algo_trading_project\backtesting_engine\src\metrics.py:36: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fillna(0)
c:\Users\yaswa\OneDrive\Documents\algo_trading_project\backtesting_engine\src\metrics.py:36: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fillna(0)



📊 Naive SMA Strategy Metrics
Sharpe Ratio: -0.33
Sortino Ratio: -0.45
Calmar Ratio: -0.98
Max Drawdown: -90.41%
Total Trades: 98, Win Rate: 28.6%
Avg Profit (Winners): 9.48%, Avg Loss (Losers): -5.84%

📊 Volatility-Aware SMA Strategy Metrics
Sharpe Ratio: -0.05
Sortino Ratio: -0.02
Calmar Ratio: -0.34
Max Drawdown: -14.39%
Total Trades: 157, Win Rate: 34.4%
Avg Profit (Winners): 4.02%, Avg Loss (Losers): -5.00%


## 📈 Strategy Evaluation Summary

### 1. Naive SMA Strategy (20/50)
- **Sharpe Ratio**: -0.33
- **Sortino Ratio**: -0.45
- **Calmar Ratio**: -0.98
- **Max Drawdown**: -90.41%
- **Total Trades**: 98
- **Win Rate**: 28.6%
- **Avg Profit (Winners)**: 9.48%
- **Avg Loss (Losers)**: -5.84%

📉 Result: The naive SMA strategy performed poorly on XLE, with large drawdowns and low win rate. It struggles in sideways or choppy markets.

---

### 2. Volatility-Aware SMA (ATR Filter)
- **Sharpe Ratio**: -0.05
- **Sortino Ratio**: -0.02
- **Calmar Ratio**: -0.34
- **Max Drawdown**: -14.39%
- **Total Trades**: 157
- **Win Rate**: 34.4%
- **Avg Profit (Winners)**: 4.02%
- **Avg Loss (Losers)**: -5.00%

🟠 Result: Adding an ATR-based filter reduced drawdowns significantly but didn’t capture the uptrend. It's more defensive and less profitable.

---

### 3. Buy & Hold (Benchmark)
✅ Outperformed both strategies with a strong upward trend post-2020.

---

## 📌 Key Takeaways:
- SMA crossovers need careful tuning and work better in trending markets.
- Risk filters like ATR can reduce drawdowns but also suppress gains.
- Strategy design must match **asset behavior**.